In [1]:
using Pkg
Pkg.activate(".")
Pkg.add("BenchmarkTools")
Pkg.add("DataFrames")

using BenchmarkTools
using DataFrames

  Activating project at `~`
    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/Project.toml`
  No Changes to `~/Manifest.toml`
   Resolving package versions...
   Installed InvertedIndices ──── v1.2.0
   Installed StringManipulation ─ v0.3.0
   Installed PrettyTables ─────── v2.2.2
   Installed DataFrames ───────── v1.4.4
    Updating `~/Project.toml`
  [a93c6f00] + DataFrames v1.4.4
    Updating `~/Manifest.toml`
  [a8cc5b0e] + Crayons v4.1.1
  [a93c6f00] + DataFrames v1.4.4
  [41ab1584] + InvertedIndices v1.2.0
  [2dfb63ee] + PooledArrays v1.4.2
  [08abe8d2] + PrettyTables v2.2.2
  [892a3eda] + StringManipulation v0.3.0
Precompiling project...
  ✓ InvertedIndices
  ✓ StringManipulation
  ✓ PrettyTables
  ✓ DataFrames
  4 dependencies successfully precompiled in 16 seconds. 281 already precompiled. 1 skipped during auto due to previous errors.


### KR 1

Benchmarked at least two (2) different implementation of the same function or process (e.g. raising each element of an array to some power p, random array may be used) that utilizes some parameter that can be considered a constant or declared globally.

In [2]:
p = 2

function raisetop(x::Vector)
    s = zero(eltype(x));
    for xel in x
        s = s + xel^p
    end
    return s
end

raisetop (generic function with 1 method)

In [3]:
data = rand(1_000_000)

1000000-element Vector{Float64}:
 0.2645312529742897
 0.0977553537289344
 0.45238047675079873
 0.9939724720772668
 0.030085011346754897
 0.19544881843144246
 0.10845753367785615
 0.8340513888473609
 0.568230059249739
 0.9701304818087785
 0.41202892731184726
 0.3690835656484285
 0.5874139942797383
 ⋮
 0.30272220058339194
 0.42967262804935624
 0.7650299180406568
 0.02598199474589935
 0.05750273800906147
 0.3452603266769694
 0.0606276302903328
 0.9578618204213883
 0.5374922606775399
 0.43005935640476356
 0.16994896805870707
 0.5763755913612664

In [4]:
mark0 = @benchmark raisetop($data)

BenchmarkTools.Trial: 127 samples with 1 evaluation.
 Range (min … max):  36.947 ms … 43.380 ms  ┊ GC (min … max): 0.00% … 5.82%
 Time  (median):     40.244 ms              ┊ GC (median):    5.56%
 Time  (mean ± σ):   39.614 ms ±  1.478 ms  ┊ GC (mean ± σ):  3.95% ± 2.75%

                                      █                        
  ▆▄▃▅▄▃▄▇▅▃▆▃▆▄▄▁▁▁▃▁▁▁▃▁▄▄▅▄▄▃▃▃▁▄▅▆█▅▆█▆▄▆▆▃▃▄▅▁▅▁▁▁▁▃▃▃▁▃ ▃
  36.9 ms         Histogram: frequency by time        42.4 ms <

 Memory estimate: 45.78 MiB, allocs estimate: 3000000.

In [6]:
@code_warntype raisetop(data)

MethodInstance for raisetop(::Vector{Float64})
  from raisetop(x::Vector) in Main at In[2]:3
Arguments
  #self#::Core.Const(raisetop)
  x::Vector{Float64}
Locals
  @_3::Union{Nothing, Tuple{Float64, Int64}}
  s::Any
  xel::Float64
Body::Nothing
1 ─ %1  = Main.eltype(x)::Core.Const(Float64)
│         (s = Main.zero(%1))
│   %3  = x::Vector{Float64}
│         (@_3 = Base.iterate(%3))
│   %5  = (@_3 === nothing)::Bool
│   %6  = Base.not_int(%5)::Bool
└──       goto #4 if not %6
2 ┄ %8  = @_3::Tuple{Float64, Int64}
│         (xel = Core.getfield(%8, 1))
│   %10 = Core.getfield(%8, 2)::Int64
│   %11 = s::Any
│   %12 = (xel ^ Main.p)::Any
│         (s = %11 + %12)
│         (@_3 = Base.iterate(%3, %10))
│   %15 = (@_3 === nothing)::Bool
│   %16 = Base.not_int(%15)::Bool
└──       goto #4 if not %16
3 ─       goto #2
4 ┄       return nothing



In [7]:
const pconst = 2;

function raisetop_const(x::Vector)
    s = zero(eltype(x));
    for xel in x
        s = s + xel^pconst
    end
    return s
end

raisetop_const (generic function with 1 method)

In [8]:
mark1 = @benchmark raisetop_const($data)

BenchmarkTools.Trial: 1767 samples with 1 evaluation.
 Range (min … max):  2.811 ms …  3.232 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     2.819 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.830 ms ± 24.564 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

    █                                                         
  ▂▅█▅▅▅▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  2.81 ms        Histogram: frequency by time         2.9 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [9]:
@code_warntype raisetop_const(data)

MethodInstance for raisetop_const(::Vector{Float64})
  from raisetop_const(x::Vector) in Main at In[7]:3
Arguments
  #self#::Core.Const(raisetop_const)
  x::Vector{Float64}
Locals
  @_3::Union{Nothing, Tuple{Float64, Int64}}
  s::Float64
  xel::Float64
Body::Nothing
1 ─ %1  = Main.eltype(x)::Core.Const(Float64)
│         (s = Main.zero(%1))
│   %3  = x::Vector{Float64}
│         (@_3 = Base.iterate(%3))
│   %5  = (@_3 === nothing)::Bool
│   %6  = Base.not_int(%5)::Bool
└──       goto #4 if not %6
2 ┄ %8  = @_3::Tuple{Float64, Int64}
│         (xel = Core.getfield(%8, 1))
│   %10 = Core.getfield(%8, 2)::Int64
│   %11 = s::Float64
│   %12 = (xel ^ Main.pconst)::Float64
│         (s = %11 + %12)
│         (@_3 = Base.iterate(%3, %10))
│   %15 = (@_3 === nothing)::Bool
│   %16 = Base.not_int(%15)::Bool
└──       goto #4 if not %16
3 ─       goto #2
4 ┄       return nothing



In [10]:
speedup1 = median(mark0.times) / median(mark1.times)

table = DataFrame("Method" => ["Global", "Constant"], "Speedup" => [1.0, speedup1]);

print(table)

2×2 DataFrame
 Row │ Method    Speedup 
     │ String    Float64 
─────┼───────────────────
   1 │ Global     1.0
   2 │ Constant  14.2742

In [16]:
function raisetop_param(x::Vector; pow=2)
    s = zero(eltype(x));
    for xel in x
        s = s + xel^pow
    end
    return s
end

raisetop_param (generic function with 2 methods)

In [17]:
mark2 = @benchmark raisetop_param($data, pow=p)

BenchmarkTools.Trial: 1762 samples with 1 evaluation.
 Range (min … max):  2.811 ms …  3.444 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     2.830 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.837 ms ± 32.759 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

   █                                                          
  ▅██▇▅▅▆▅▆▆▆▅▅▅▅▄▃▃▃▃▃▂▃▃▂▃▂▃▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂▂▂▂▂▂▂▂▁▁▂▂▁▂▂▂ ▃
  2.81 ms        Histogram: frequency by time        2.96 ms <

 Memory estimate: 48 bytes, allocs estimate: 3.

In [25]:
@code_warntype raisetop_param(data; pow=p)

MethodInstance for (::var"#raisetop_param##kw")(::NamedTuple{(:pow,), Tuple{Int64}}, ::typeof(raisetop_param), ::Vector{Float64})
  from (::var"#raisetop_param##kw")(::Any, ::typeof(raisetop_param), x::Vector) in Main at In[16]:1
Arguments
  _::Core.Const(var"#raisetop_param##kw"())
  @_2::NamedTuple{(:pow,), Tuple{Int64}}
  @_3::Core.Const(raisetop_param)
  x::Vector{Float64}
Locals
  pow::Int64
  @_6::Int64
Body::Float64
1 ─ %1  = Base.haskey(@_2, :pow)::Core.Const(true)
│         Core.typeassert(%1, Core.Bool)
│         (@_6 = Base.getindex(@_2, :pow))
└──       goto #3
2 ─       Core.Const(:(@_6 = 2))
3 ┄ %6  = @_6::Int64
│         (pow = %6)
│   %8  = (:pow,)::Core.Const((:pow,))
│   %9  = Core.apply_type(Core.NamedTuple, %8)::Core.Const(NamedTuple{(:pow,)})
│   %10 = Base.structdiff(@_2, %9)::Core.Const(NamedTuple())
│   %11 = Base.pairs(%10)::Core.Const(Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}}())
│   %12 = Base.isempty(%11)::Core.Const(true)
│         Core.t

In [18]:
mark2a = @benchmark raisetop_param($data, pow=2)

BenchmarkTools.Trial: 1761 samples with 1 evaluation.
 Range (min … max):  2.811 ms …  3.095 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     2.832 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.838 ms ± 25.474 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

   █▁     ▁ ▁                                                 
  ▅██▇█▇█▇██████▇▆▅▄▄▅▄▄▄▅▃▄▃▃▄▃▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▂▁▂▁▂▂▁▂ ▃
  2.81 ms        Histogram: frequency by time        2.92 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [26]:
@code_warntype raisetop_param(data; pow=2)

MethodInstance for (::var"#raisetop_param##kw")(::NamedTuple{(:pow,), Tuple{Int64}}, ::typeof(raisetop_param), ::Vector{Float64})
  from (::var"#raisetop_param##kw")(::Any, ::typeof(raisetop_param), x::Vector) in Main at In[16]:1
Arguments
  _::Core.Const(var"#raisetop_param##kw"())
  @_2::NamedTuple{(:pow,), Tuple{Int64}}
  @_3::Core.Const(raisetop_param)
  x::Vector{Float64}
Locals
  pow::Int64
  @_6::Int64
Body::Float64
1 ─ %1  = Base.haskey(@_2, :pow)::Core.Const(true)
│         Core.typeassert(%1, Core.Bool)
│         (@_6 = Base.getindex(@_2, :pow))
└──       goto #3
2 ─       Core.Const(:(@_6 = 2))
3 ┄ %6  = @_6::Int64
│         (pow = %6)
│   %8  = (:pow,)::Core.Const((:pow,))
│   %9  = Core.apply_type(Core.NamedTuple, %8)::Core.Const(NamedTuple{(:pow,)})
│   %10 = Base.structdiff(@_2, %9)::Core.Const(NamedTuple())
│   %11 = Base.pairs(%10)::Core.Const(Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}}())
│   %12 = Base.isempty(%11)::Core.Const(true)
│         Core.t

In [20]:
mark2b = @benchmark raisetop_param($data, pow=pconst)

BenchmarkTools.Trial: 1760 samples with 1 evaluation.
 Range (min … max):  2.811 ms …  3.085 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     2.833 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.840 ms ± 25.424 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▇█▄▄▄▅▆▆▅▅▄▃▃                                              
  ▅██████████████▆▇▇▆▆▆▅▅▆▅▆▄▅▅▄▅▄▄▄▄▃▃▃▃▃▃▂▂▂▂▃▃▂▂▂▂▂▂▂▂▂▁▂ ▄
  2.81 ms        Histogram: frequency by time        2.93 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [27]:
@code_warntype raisetop_param(data; pow=pconst)

MethodInstance for (::var"#raisetop_param##kw")(::NamedTuple{(:pow,), Tuple{Int64}}, ::typeof(raisetop_param), ::Vector{Float64})
  from (::var"#raisetop_param##kw")(::Any, ::typeof(raisetop_param), x::Vector) in Main at In[16]:1
Arguments
  _::Core.Const(var"#raisetop_param##kw"())
  @_2::NamedTuple{(:pow,), Tuple{Int64}}
  @_3::Core.Const(raisetop_param)
  x::Vector{Float64}
Locals
  pow::Int64
  @_6::Int64
Body::Float64
1 ─ %1  = Base.haskey(@_2, :pow)::Core.Const(true)
│         Core.typeassert(%1, Core.Bool)
│         (@_6 = Base.getindex(@_2, :pow))
└──       goto #3
2 ─       Core.Const(:(@_6 = 2))
3 ┄ %6  = @_6::Int64
│         (pow = %6)
│   %8  = (:pow,)::Core.Const((:pow,))
│   %9  = Core.apply_type(Core.NamedTuple, %8)::Core.Const(NamedTuple{(:pow,)})
│   %10 = Base.structdiff(@_2, %9)::Core.Const(NamedTuple())
│   %11 = Base.pairs(%10)::Core.Const(Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}}())
│   %12 = Base.isempty(%11)::Core.Const(true)
│         Core.t

In [21]:
speedup2 = median(mark0.times) / median(mark2.times)
speedup2a = median(mark0.times) / median(mark2a.times)
speedup2b = median(mark0.times) / median(mark2b.times)

push!(table, ["Parametrized", speedup2]);
push!(table, ["Parametrized const.exp", speedup2a]);
push!(table, ["Parametrized const.imp", speedup2b]);


print(table)

5×2 DataFrame
 Row │ Method                  Speedup 
     │ String                  Float64 
─────┼─────────────────────────────────
   1 │ Global                   1.0
   2 │ Constant                14.2742
   3 │ Parametrized            14.2206
   4 │ Parametrized const.exp  14.2093
   5 │ Parametrized const.imp  14.2068

In [29]:
function raisetop_map(data; pow=pconst)
    raised = zeros(size(data))
    map!(x->x^pow, raised, data)
    return sum(raised)
end

raisetop_map (generic function with 1 method)

In [30]:
mark3 = @benchmark raisetop_map($data; pow=pconst)

BenchmarkTools.Trial: 1373 samples with 1 evaluation.
 Range (min … max):  3.071 ms …   7.643 ms  ┊ GC (min … max): 0.00% …  0.00%
 Time  (median):     3.455 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   3.640 ms ± 539.660 μs  ┊ GC (mean ± σ):  4.73% ± 10.03%

          ▆█▆▄▁                                                
  █▄▁▁▁▁▁▁████████▇▆▅▄▄▁▅▄▄▄▁▄▁▁▁▁▁▁▁▁▁▄▁▄▄▁▁▄▆▆▇▆▇██▇▇██▇▆▅▇ █
  3.07 ms      Histogram: log(frequency) by time      5.39 ms <

 Memory estimate: 7.63 MiB, allocs estimate: 2.

In [31]:
speedup3 = median(mark0.times) / median(mark3.times)

push!(table, ["Base.mapped", speedup3])

print(table)

6×2 DataFrame
 Row │ Method                  Speedup 
     │ String                  Float64 
─────┼─────────────────────────────────
   1 │ Global                   1.0
   2 │ Constant                14.2742
   3 │ Parametrized            14.2206
   4 │ Parametrized const.exp  14.2093
   5 │ Parametrized const.imp  14.2068
   6 │ Base.mapped             11.6474

### KR2: 

Replicated the naive implementation of the polynomial in the textbook.

In [32]:
function poly_naive(x, a...)
    p = zero(x)
    for i in eachindex(a)
        p = p + a[i] * x^(i-1)
    end
    return p
end

poly_naive (generic function with 1 method)

In [33]:
f_naive(x) = poly_naive(x, 1,2,3,4,5,6,7,8,9)

f_naive (generic function with 1 method)

In [34]:
x = 3.5

3.5

In [35]:
mark0 = @benchmark f_naive($x)

BenchmarkTools.Trial: 10000 samples with 995 evaluations.
 Range (min … max):  29.271 ns … 54.104 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     30.570 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   30.219 ns ±  1.076 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

   █                 ▄▅                                        
  ▃█▄▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅██▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  29.3 ns         Histogram: frequency by time        33.3 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

### KR3: 

Replicated the naive implementation of the Horner's method for the same polynomial.

In [36]:
function poly_h(x, a...)
    b = zero(x)
    for i in reverse(eachindex(a))
        b = a[i] * b*x
    end
    return b
end     

poly_h (generic function with 1 method)

In [37]:
f_h(x) = poly_h(x, 1,2,3,4,5,6,7,8,9)

f_h (generic function with 1 method)

In [38]:
x = 3.5

3.5

In [39]:
mark1 = @benchmark f_h($x)

BenchmarkTools.Trial: 10000 samples with 1000 evaluations.
 Range (min … max):  2.417 ns … 16.584 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     2.541 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.537 ns ±  0.229 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

             ▁           ▇          █           ▃          ▃ ▁
  ▃▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁█ █
  2.42 ns      Histogram: log(frequency) by time     2.62 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [41]:
speedup1 = median(mark0.times) / median(mark1.times)

table = DataFrame("Method" => ["Global", "Constant"], "Speedup" => [1.0, speedup1])

print(table)

2×2 DataFrame
 Row │ Method    Speedup 
     │ String    Float64 
─────┼───────────────────
   1 │ Global     1.0
   2 │ Constant  12.0306

### KR4: 

Replicated the macro implementation of the Horner's method of the same polynomial.

In [53]:
macro horner(x, p...)
    ex = esc(p[end])
    for i in length(p)-1:-1:1
        ex = :(muladd(t, $(ex), $(esc(p[i]))))
    end
    Expr(:block, :(t=$(esc(x))), ex)
end

@horner (macro with 1 method)

In [57]:
f_h_macro(x) = @horner(x, 1,2,3,4,5,6,7,8,9)

f_h_macro (generic function with 1 method)

In [58]:
mark2 = @benchmark f_h_macro($x)

BenchmarkTools.Trial: 10000 samples with 1000 evaluations.
 Range (min … max):  2.416 ns … 30.958 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     2.500 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.530 ns ±  0.366 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

             ▂           █          █           ▂            ▂
  ▅▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁█ █
  2.42 ns      Histogram: log(frequency) by time     2.62 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [59]:
speedup2 = median(mark0.times) / median(mark2.times)

push!(table, ["Macro", speedup2]);

print(table)

3×2 DataFrame
 Row │ Method    Speedup 
     │ String    Float64 
─────┼───────────────────
   1 │ Global     1.0
   2 │ Constant  12.0306
   3 │ Macro     12.2279

### KR5: 

Table showing how many minutes will the function evaluations in both KR3 and KR4 be reduced if KR2 requires 24hours of runtime.

In [60]:
for r in eachrow(table)
    println("$(24*60/r.Speedup) mins for $(r.Method) method.")
end

1440.0 mins for Global method.
119.69440773251802 mins for Constant method.
117.76309300719993 mins for Macro method.


In [61]:
transform!(table, :Speedup => ByRow(x->24*60/x)=>:"Time(mins)")
print(table)

3×3 DataFrame
 Row │ Method    Speedup  Time(mins) 
     │ String    Float64  Float64    
─────┼───────────────────────────────
   1 │ Global     1.0       1440.0
   2 │ Constant  12.0306     119.694
   3 │ Macro     12.2279     117.763